In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import wptools
import wikipedia
import re
import numpy as np

from prefect import task, Flow, Parameter
from prefect.executors import LocalDaskExecutor
import nest_asyncio
nest_asyncio.apply()

#### web scrape company name, policy, link, story

In [2]:
url = 'https://buildremote.co/companies/companies-going-remote-permanently/'
user_agent = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'

raw = requests.get(url, headers={'User-Agent': user_agent})
content_bs4 = BeautifulSoup(raw.content)

In [3]:
table_content = content_bs4.find_all('td', class_ = 'wptb-cell')
company_table = [i.get_text() for i in table_content]
company_table = company_table[4:]

In [5]:
link_content = table_content[7:]

link_list = []
for i, j in enumerate(link_content):
    if i%4 == 0:
        try:
            a_link = j.find(href= True)['href']
        except:
            a_link = ''
        
        link_list.append(a_link)

In [6]:
name = []
policy = []
story = []

for i, j in enumerate(company_table):
    if i%4 ==0:
        name.append(j.strip())
    elif i%4 == 1:
        policy.append(j)
    elif i%4 ==3:
        story.append(j)

In [7]:
output = pd.DataFrame()
output['name'] = name
output['policy'] = policy
output['link'] = link_list
output['story'] = story

#### fetch employee size

In [8]:
#### use wptools to filter not found + wikipedia.search

def get_wiki_disambiguation(clean_name):
    get_data = wptools.page(clean_name).get()
    if get_data is not None:
        if 'what' in get_data.data.keys():
            if get_data.data['what'] == 'Wikimedia disambiguation page':
                links = get_data.data['links']
                for i in links:
                    check_list = ['name', 'logo', 'type', 'traded_as', 'foundation', 'founded', 'released', 'num_employees', 'num_staff']
                    so = wptools.page(i).get_parse()
                    if 'infobox' in so.data.keys():
                        infobox = so.data['infobox']
                        if infobox is not None:
                            intersection = [value for value in check_list if value in infobox.keys()]
                            if len(intersection) >= 3:
                                break
                        else:
                            infobox = {'error': 'Wikipedia page Not Found'}
                    else:
                            infobox = {'error': 'Wikipedia page Not Found'}

            else:
                so = wptools.page(clean_name).get_parse()
                infobox = so.data['infobox']
        else:
            so = wptools.page(clean_name).get_parse()
            infobox = so.data['infobox']
    
    else:
        infobox = infobox = {'error': 'Wikipedia page Not Found'}

    if infobox is not None:        
        infobox['clean_name'] = clean_name
    else:
        infobox = {'error': 'Wikipedia page Not Found', 'clean_name': clean_name}
    
    return infobox


def get_wiki_suggest(clean_name):
    suggest = wikipedia.search(clean_name)
    if len(suggest)>0:
        suggest_name = suggest[0]
    else:
        suggest_name = clean_name
    return suggest_name



def get_wiki_improve(clean_name):
    try:
        infobox = get_wiki_disambiguation(clean_name)
        return infobox
    except:
        try:
            suggest_name = get_wiki_suggest(clean_name)
            infobox = get_wiki_disambiguation(suggest_name)
            return infobox

        except Exception as exc:
            return {'error': exc, 'clean_name': clean_name}

## employee size
def find_size(df, columnname):
    if 'num_employees' in columnname and df['num_employees'] is not np.nan:
        size = df['num_employees']
        employee_size = size
    elif 'num_staff' in columnname and df['num_staff'] is not np.nan:
        size = df['num_staff']
        employee_size = size
    else:
        employee_size = np.nan
    return employee_size

def process_size(size):
    try:
        size = str(size).replace(',', '')

        match = re.findall(r'\d+', size)
        res = list(map(int, match))
        return res[0]
    except:
        return np.nan

In [12]:
#utility function for getting flow results
def show_result(flow, run):
    last_task = list(flow.terminal_tasks())[0]
    print(list(flow.terminal_tasks()))
    print(last_task)
    return run.result[last_task].result

def show_dataframe_task(output):
    return pd.DataFrame(output)

def merge_dataframe(df1, df2):
    return pd.concat([df1, df2],axis = 1)

In [13]:
new_company =  output.rename(columns = {'name':'refined_name'})

with Flow('clean name and get wiki improve1', executor=LocalDaskExecutor()) as wiki_flow_improve1:
    refined_name = Parameter('refined_name')
    result1 = task(get_wiki_improve).map(refined_name)
    task(show_dataframe_task)(result1)

### run on batches to get wiki result

batch_num = int(len(new_company)/600)
for i in range(batch_num):
    globals()['batch_%s' % str(i+1)] = new_company[i*600:(i+1)*600].reset_index(drop=True)
globals()['batch_%s' % str(batch_num+1)] = new_company[(batch_num)*600:].reset_index(drop=True)


for i in range(1, batch_num+2):
    current_batch = globals()['batch_'+str(i)]
    namelist = current_batch[current_batch['refined_name'].notna()]['refined_name'].tolist()
    output = wiki_flow_improve1.run(refined_name = namelist)
    globals()['wiki_res_%s' % str(i)] = show_result(wiki_flow_improve1, output)


res_list = []
for i in range(1, batch_num+2):
    res_list.append(globals()['wiki_res_%s' % str(i)])

wiki_result = pd.concat(res_list).reset_index(drop=True)

[2022-01-20 10:07:32-0500] INFO - prefect.FlowRunner | Beginning Flow run for 'clean name and get wiki improve1'
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'refined_name': Starting task run...
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'refined_name': Finished task run for task with final state: 'Success'
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve': Starting task run...
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve': Finished task run for task with final state: 'Mapped'
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[1]': Starting task run...
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[0]': Starting task run...
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[10]': Starting task run...
[2022-01-20 10:07:33-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[11]': Starting task run...


en.wikipedia.org (query) Twitter
en.wikipedia.org (query) Nationwide
en.wikipedia.org (query) Slack
en.wikipedia.org (query) Quora
en.wikipedia.org (parse) 223400
en.wikipedia.org (parse) 1322074
en.wikipedia.org (parse) 26749224
en.wikipedia.org (query) Twitter (&plcontinue=9988187|0|Lytro)
www.wikidata.org (wikidata) Q1904950
www.wikidata.org (wikidata) Q1216768
www.wikidata.org (wikidata) Q51711
www.wikidata.org (labels) Q4167410|P31|P1889|Q37555651|P373
www.wikidata.org (labels) Q4167410|P31
en.wikipedia.org (query) Twitter (&plcontinue=9988187|0|Web_appli...
www.wikidata.org (labels) Q9027|P4411|P31|P2003|P646|Q9035|P3553|...
en.wikipedia.org (restbase) /page/summary/Slack
en.wikipedia.org (restbase) /page/summary/Nationwide
Slack (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Disambiguation
  claims: <dict(3)> P31, P1889, P373
  description: Topics referred to by the same term
  disambiguation: 25
  exhtml: <p><b>Slack</b> may refer to:</p>
  exrest: Slack ma

[2022-01-20 10:07:36-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[11]': Finished task run for task with final state: 'Success'
[2022-01-20 10:07:36-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[12]': Starting task run...


George Slack (en) data
{
  pageid: 46797209
  parsetree: <str(1928)> <root>'''George W. Slack''' (April 2, 187...
  requests: <list(1)> parse
  title: George Slack
  wikibase: Q19975228
  wikidata_url: https://www.wikidata.org/wiki/Q19975228
  wikitext: <str(1495)> '''George W. Slack''' (April 2, 1874 &ndas...
}
en.wikipedia.org (parse) Resource slack
Quora (en) data
{
  infobox: <dict(20)> name, logo, collapsible, collapsetext, scree...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:Q...
  pageid: 26749224
  parsetree: <str(73307)> <root><template><title>Short description...
  requests: <list(1)> parse
  title: Quora
  wikibase: Q51711
  wikidata_url: https://www.wikidata.org/wiki/Q51711
  wikitext: <str(55939)> {{Short description|Question-and-answer p...
}
en.wikipedia.org (query) Fujitsu
Resource slack (en) data
{
  pageid: 42417759
  parsetree: <str(8135)> <root>'''Resource slack''', in the [[busi...
  requests: <list(1)> parse
  title: Resource slack
  wikibase:

[2022-01-20 10:07:37-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[0]': Finished task run for task with final state: 'Success'
[2022-01-20 10:07:37-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[13]': Starting task run...


...

Robin Hood (1912 film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Robin Hoo...
  infobox: <dict(12)> name, image, caption, director, producer, wr...
  pageid: 15110213
  parsetree: <str(5776)> <root><template><title>short description<...
  requests: <list(2)> parse, imageinfo
  title: Robin Hood (1912 film)
  wikibase: Q3939269
  wikidata_url: https://www.wikidata.org/wiki/Q3939269
  wikitext: <str(3989)> {{short description|1912 film}}{{Infobox f...
}
en.wikipedia.org (parse) Robin Hood (1922 film)
www.wikidata.org (labels) Q7014241|Q1196955|P856|P2002|P910|P1451...
VMware (en) data
{
  infobox: <dict(20)> name, logo, image_caption, type, traded_as, ...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:VMware
  pageid: 312018
  parsetree: <str(76940)> <root><template><title>short description...
  requests: <list(1)> parse
  title: VMware
  wikibase: Q11407
  wikidata_url: https://www.wikidata.org/wiki/Q11407
  wikitext: <str(58217)> {{short des

[2022-01-20 10:08:30-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[9]': Finished task run for task with final state: 'Success'


en.wikipedia.org (imageinfo) File:Robin Hood (1991 film) cover.jpg
Robin Hood (1991 British film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Robin Hoo...
  infobox: <dict(18)> name, image, caption, director, producer, sc...
  pageid: 6309072
  parsetree: <str(11699)> <root><template><title>for</title><part>...
  requests: <list(2)> parse, imageinfo
  title: Robin Hood (1991 British film)
  wikibase: Q689658
  wikidata_url: https://www.wikidata.org/wiki/Q689658
  wikitext: <str(8304)> {{for|the 1991 film starring Kevin Costner...
}
en.wikipedia.org (parse) Robin Hood (2006 TV series)
en.wikipedia.org (imageinfo) File:Robinhoods2titlescreen.jpg
Robin Hood (2006 TV series) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Robinhood...
  infobox: <dict(21)> name, image, caption, genre, creator, starri...
  iwlinks: <list(1)> https://en.wikiquote.org/wiki/Robin_Hood_(BBC)
  pageid: 4017773
  parsetree: <str(31340)> <root><template><title>Use dmy 

[2022-01-20 10:08:35-0500] INFO - prefect.TaskRunner | Task 'get_wiki_improve[70]': Finished task run for task with final state: 'Success'
[2022-01-20 10:08:36-0500] INFO - prefect.TaskRunner | Task 'show_dataframe_task': Starting task run...
[2022-01-20 10:08:36-0500] INFO - prefect.TaskRunner | Task 'show_dataframe_task': Finished task run for task with final state: 'Success'
[2022-01-20 10:08:36-0500] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[<Task: show_dataframe_task>]
<Task: show_dataframe_task>


Robin Hood (album) (en) data
{
  infobox: <dict(8)> name, type, artist, released, genre, length, ...
  pageid: 8890924
  parsetree: <str(3899)> <root><template><title>Unreferenced</titl...
  requests: <list(1)> parse
  title: Robin Hood (album)
  wikibase: Q7352492
  wikidata_url: https://www.wikidata.org/wiki/Q7352492
  wikitext: <str(2340)> {{Unreferenced|date=January 2007}}{{Infobo...
}


In [15]:
new_company = pd.concat([new_company.reset_index(drop = True), wiki_result], axis=1)
columnname = new_company.columns
new_company['size_wiki'] = new_company.apply(lambda x: find_size(x, columnname),axis=1).apply(process_size)

In [18]:
new_company = new_company[['refined_name','policy','link','story','size_wiki']]

In [23]:
new_company.groupby('policy').count()

,refined_name,link,story,size_wiki
policy,,,,
Fully Remote,3,3,3,1
Hybrid/Optional,2,2,2,2
Hybrid/Partial,6,6,6,4
Optional,13,13,13,11
Optional/Partial,1,1,1,1
Partial,17,17,17,14
Partial/Optional,2,2,2,2
Partial/Remote First,1,1,1,1
Partial,1,1,1,1


In [21]:
# new_company.to_excel('Companies with Remote Programs JH Edit.xlsx')